In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.1'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [9]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [13]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
    SELECT year(date) AS sales_year, ROUND(AVG(price),2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY sales_year
    ORDER BY sales_year
"""

average_prices_per_year = spark.sql(query)
average_prices_per_year.show()


+----------+---------+
|sales_year|avg_price|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+



In [15]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
    SELECT year(date) AS sales_year, ROUND(AVG(price),2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY sales_year
    ORDER BY sales_year
"""

average_prices_per_year_2 = spark.sql(query)
average_prices_per_year_2.show()

+----------+---------+
|sales_year|avg_price|
+----------+---------+
|      2019|287287.82|
|      2020|294204.16|
|      2021|294211.46|
|      2022|292725.69|
+----------+---------+



In [17]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query = """
    SELECT year(date) AS sales_year, ROUND(AVG(price),2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY sales_year
    ORDER BY sales_year
"""

average_prices_per_year_3 = spark.sql(query)
average_prices_per_year_3.show()


+----------+---------+
|sales_year|avg_price|
+----------+---------+
|      2019|289859.14|
|      2020|292289.09|
|      2021|296330.96|
|      2022|290242.99|
+----------+---------+



In [18]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
"""

average_price_by_view = spark.sql(query)
average_price_by_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  16| 399586.53|
+----+----------+
only showing top 20 rows

--- 0.8678970336914062 seconds ---


In [19]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [22]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query_cached = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
"""

average_price_by_view_cached = spark.sql(query_cached)
average_price_by_view_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  16| 399586.53|
+----+----------+
only showing top 20 rows

--- 1.0086076259613037 seconds ---


In [23]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
output_path = "/Home_Sales"
df.write.partitionBy("date_built").parquet(output_path)

In [24]:
# 11. Read the parquet formatted data.
parquet_data = spark.read.parquet("/Home_Sales")

# Show the DataFrame
parquet_data.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [26]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_data_temp")


In [28]:
# Query the temporary view
query = spark.sql("SELECT * FROM parquet_data_temp LIMIT 10").show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [30]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query_parquet = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM parquet_data_temp
    WHERE price >= 350000
    GROUP BY view
"""

average_price_by_view_parquet = spark.sql(query_parquet)
average_price_by_view_parquet.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  73| 752861.18|
|  87| 1072285.2|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  35| 401934.21|
+----+----------+
only showing top 20 rows

--- 0.8470921516418457 seconds ---


In [31]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [32]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False

In [33]:
# Stop the Spark session
spark.stop()